In [1]:
#import le librerie
import os
import  shutil
import csv

In [2]:
#mi sposto nella cartella files
os.chdir('files')

In [4]:
#creo le sottocartelle
for dirname in ['images', 'docs', 'audio']:
    if not os.path.exists(dirname):
        os.makedirs(dirname)

#creo dizionario con le liste dei formati accettati
dict_file = {'img_ext': ['.png', '.jpeg', '.jpg'], 'doc_ext': ['.txt', '.odt'], 'audio_ext': ['.mp3']}


#se il file di recup non esiste lo creo e scrivo gli header del csv
if not os.path.exists('recap.csv'):
    fields = ['name', 'type', 'size(B)']
    with open('recap.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields)

else: #se recup.csv esiste già lo apro in append mode         
    with open('recap.csv', 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)

        #itero nella cartella files
        for f in sorted(os.listdir('.')):
            
            #separo nome file ed estensione
            name, ext = os.path.splitext(f)
            
            if ext in dict_file['img_ext']:#verifico estensione
                size = os.path.getsize(f)#calcolo le dimensioni del file
                print(f'{name} type:image size:{size}B')#stampo le info del file
                csvwriter.writerow([name, 'image', size])#appendo le info del file a recup.csv
                shutil.move(f, 'images/')#sposto il file nella relativa sottocartella

            elif ext in dict_file['doc_ext']:
                size = os.path.getsize(f)
                print(f'{name} type:doc size:{size}B')
                csvwriter.writerow([name, 'doc', size])
                shutil.move(f, 'docs/')

            elif ext in dict_file['audio_ext']:
                size = os.path.getsize(f)
                print(f'{name} type:audio size:{size}B')
                csvwriter.writerow([name, 'audio', size])
                shutil.move(f, 'audio/')

            else:
                if os.path.isfile(f):
                    print(f'Not accepted format for file {f}')
                else:
                    pass


In [5]:
from PIL import Image
from tabulate import tabulate
import numpy as np

In [6]:
table = []#creo lista per la tabella

#itero nella cartella images        
for img_name in sorted(os.listdir('images')):
    #creo il percorso per caricare l'immaggine
    img = Image.open(os.path.join('images', img_name))
    #creo l'array dell'immaggine
    img_array = np.array(img)
    
    #calcolo altezza e larghezza
    pixel_y, pixel_x = img_array.shape[:2]
    
    #caso scala di grigio
    if img_array.ndim == 2:
        mean_gs = img_array.mean()#media dei valori dell'unico livello di colore
        mean_r, mean_g, mean_b = 0, 0, 0 #non esistono altri lvelli
        mean_alpha = 0
    
    #caso RGB o RGBA: ho una dimensione aggiuntiva per i canali di colore (e quello di trasparenza)
    else:
        mean_gs = 0
        
        channel_mean = img_array.mean(axis=0).mean(axis=0)
        if len(channel_mean) == 3:#caso RGB con solo i tre livelli di colore
            mean_r, mean_g, mean_b = channel_mean
            mean_alpha = 0
        else:
            mean_r, mean_g, mean_b, mean_alpha = channel_mean
    
    
    #separo nome ed estensione
    name, ext = os.path.splitext(img_name)
    #creo gli argomenti della lista table
    row = [name, pixel_y, pixel_x, mean_gs, mean_r, mean_g, mean_b, mean_alpha]
    
    #aggiungo riga atabella
    table += [row]

#stampo tabella con tabulate
print(tabulate(table, headers=['name', 'height', 'width', 'grayscale', 'R', 'G', 'B', 'ALPHA'],
               tablefmt='fancy_grid', numalign='right', floatfmt='.2f'))




╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
